# Customizing Models and training algorithms

## Loss Function
En algunos casos quizás por las características de los datos, sea necesario utilizar funciones de costo alternativas, como en este ejemplo donde se simula la implementación de Huber loss function a pesar de que keras contiene tf.keras.losses.Huber. 

En un caso hipotético donde se entrena un modelo de regresión, y luego de realizar la limpieza de outliers de los datos, aún mantienen cierto ruido, por lo cual el error cuadratico medio puede penalizar demasiado a errores largos y generar un modelo impreciso. El error absoluto medio quizás no penaliza tanto a los outliers, pero el entrenamiento pueden demorar en converger, y el modelo entrenado puede ser impreciso también. Por lo tanto se procede a usar una función de pérdida custom en lugar de MSE.


In [2]:
import tensorflow as tf

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    max_error = 1
    is_small_error = tf.abs(error) < max_error
    squared_loss = tf.square(error)
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

Para mejor performance se debe utilizar únicamente una implementación vectorial, para beneficiarse de la optimización de TensorFlow solo se deben usar operaciones de TensorFlow

In [3]:
@tf.function
def huber_fn(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = tf.abs(error)
    quadratic = 0.5 * tf.square(error)
    linear = delta * (abs_error - 0.5 * delta)
    return tf.where(abs_error <= delta, quadratic, linear)

- Vectorización total: todo opera sobre tensores, sin bucles.
- Multiplica por delta en la parte lineal (más general y matemáticamente correcto).
- Decorador @tf.function: compila la función en una graph function, mejorando performance al ser usado en entrenamiento o producción.